In [1]:
import os
import glob

import pandas as pd
import numpy as np
np.set_printoptions(precision=6, suppress=True)
from sklearn.utils import shuffle

from tqdm import tqdm

import tensorflow as tf
from tensorflow.keras import *
import tensorflow_addons as tfa
tf.__version__

KeyboardInterrupt: 

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import (LinearLocator, MultipleLocator, FormatStrFormatter)
from matplotlib.dates import MONDAY
from matplotlib.dates import MonthLocator, WeekdayLocator, DateFormatter
from matplotlib import gridspec
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = ((8/2.54), (6/2.54))
plt.rcParams["font.family"] = "Arial"
plt.rcParams["mathtext.default"] = "rm"
plt.rcParams.update({'font.size': 11})
MARKER_SIZE = 15
cmap_m = ["#f4a6ad", "#f6957e", "#fccfa2", "#8de7be", "#86d6f2", "#24a9e4", "#b586e0", "#d7f293"]
cmap = ["#e94d5b", "#ef4d28", "#f9a54f", "#25b575", "#1bb1e7", "#1477a2", "#a662e5", "#c2f442"]

plt.rcParams['axes.spines.top'] = False
# plt.rcParams['axes.edgecolor'] = 
plt.rcParams['axes.linewidth'] = 1
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.major.width'] = 1
plt.rcParams['ytick.minor.width'] = 1

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [ ]:
strategy = tf.distribute.MirroredStrategy()

# Hyperparameters

In [ ]:
BEST_PATH = './models/Transformer.h5'
TRAINING_EPOCHS = 200
LEARNING_RATE = 0.002
EPSILON = 1e-06
BATCH_SIZE = 16

# Data loading

In [ ]:
l = np.load('./results/2020_S/modeling_dataset.npz', allow_pickle=True)
output_columns = l['output_columns']
input_indices = l['input_indices']
output_indices = l['output_indices']
input_data = l['input_data']
output_label = l['output_label']
INPUT_MAXS = l['INPUT_MAXS']
INPUT_MINS = l['INPUT_MINS']
OUTPUT_MAXS = l['OUTPUT_MAXS']
OUTPUT_MINS = l['OUTPUT_MINS']

In [ ]:
input_data = input_data.astype('float32')
output_label = output_label.astype('float32')

In [ ]:
input_indices = input_indices.reshape(input_data.shape[:2])

In [ ]:
print(input_data.shape)
print(output_label.shape)

In [ ]:
input_indices, output_indices, input_data, output_label = shuffle(input_indices, output_indices, input_data, output_label, random_state=3101)

In [ ]:
N_TRAIN = int(input_data.shape[0]*.7)
train_input = input_data[:N_TRAIN, ...]
train_label = output_label[:N_TRAIN, ...]
train_indices = output_indices[:N_TRAIN]
val_input = input_data[N_TRAIN:, ...]
val_label = output_label[N_TRAIN:, ...]
val_indices = output_indices[N_TRAIN:]

In [ ]:
print(f'number of training set: {train_input.shape[0]}')
print(f'number of validation set: {val_input.shape[0]}')

In [ ]:
with strategy.scope():
    train_dataset = tf.data.Dataset.from_tensor_slices((train_input, train_label))
    train_dataset = train_dataset.cache().shuffle(BATCH_SIZE*10).batch(BATCH_SIZE, drop_remainder=False)
    val_dataset = tf.data.Dataset.from_tensor_slices((val_input, val_label))
    val_dataset = val_dataset.cache().shuffle(BATCH_SIZE*10).batch(BATCH_SIZE, drop_remainder=False)

In [ ]:
print(train_input.shape)
print(train_label.shape)

# Model construction

In [ ]:
class EmbeddingLayer(layers.Layer):
    def __init__(self, num_nodes):
        super(EmbeddingLayer, self).__init__()
        self.n = num_nodes
        
        self.dense = layers.Dense(self.n)
        self.norm = layers.LayerNormalization(epsilon=1e-6)
        
    def call(self, inp, is_train=True, **kwargs):
        
        inp = self.dense(inp)
        inp = self.norm(inp, training=is_train)

        return inp

In [ ]:
class EncoderBlock(layers.Layer):
    def __init__(self, num_nodes, num_heads):
        super(EncoderBlock, self).__init__()
        self.n = num_nodes
        self.h = num_heads
        self.d = self.n // self.h
        
        self.wq = layers.Dense(self.n)
        self.wk = layers.Dense(self.n)
        self.wv = layers.Dense(self.n)
        
        self.dropout = layers.Dropout(0.1)
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        
        self.dense1 = layers.Dense(self.n, activation=tf.nn.relu)
        self.dense2 = layers.Dense(self.n)
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)
        
    def head_maker(self, x, axis_1=2, axis_2=0):
        x = tf.concat(tf.split(x, self.h, axis=axis_1), axis=axis_2)
        
        return x
    
    def call(self, inp, is_train=True, **kwargs):
        Q = self.head_maker(self.wq(inp))
        K = self.head_maker(self.wk(inp))
        V = self.head_maker(self.wv(inp))
        
        oup = tf.matmul(Q, tf.transpose(K, (0, 2, 1)))
        oup = oup / tf.math.sqrt(tf.cast(K.shape[-1], tf.float32))
        oup = tf.nn.softmax(oup)
        oup = self.dropout(oup, training=is_train)
        oup = tf.matmul(oup, V)
        oup = self.head_maker(oup, 0, 2)
        oup += inp
        oup = self.norm1(oup, training=is_train)
        
        oup_ffnn = self.dense1(oup)
        oup_ffnn = self.dense2(oup_ffnn)
        oup += oup_ffnn
        oup = self.norm2(oup, training=is_train)
        
        return oup

In [ ]:
class TransformerLike(Model):
    def __init__(self, num_nodes, num_heads, num_layers):
        super(TransformerLike, self).__init__()
        self.n = num_nodes
        self.h = num_heads
        self.l = num_layers
        
        self.emb = EmbeddingLayer(self.n)
        self.encs = [EncoderBlock(self.n, self.h) for _ in range(self.l)]
        
        self.dense1 = layers.Dense(self.n, activation=tf.nn.relu)
        self.dense2 = layers.Dense(self.n, activation=tf.nn.relu)
        self.dense3 = layers.Dense(self.n, activation=tf.nn.relu)
        self.flatten = layers.Flatten()
        self.outdense = layers.Dense(11)        
        
    def call(self, inp, is_train=True, **kwargs):
        
        inp = self.emb(inp)
        for i in range(self.l):
            inp = self.encs[i](inp, training=is_train)
            
        inp = self.dense1(inp)
        inp = self.dense2(inp)
        inp = self.dense3(inp)
        inp = self.outdense(self.flatten(inp))
        
        return inp

In [ ]:
callbacks = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=.5, patience=2, verbose=0, mode='min',
    min_delta=0.0001, cooldown=0, min_lr=0)

save = tf.keras.callbacks.ModelCheckpoint(
    BEST_PATH, monitor='val_loss', verbose=0,
    save_best_only=True, save_weights_only=True, mode='min', save_freq='epoch')

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=20) 

In [ ]:
with strategy.scope():
    opt = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE, epsilon=EPSILON)
    model = TransformerLike(128, 4, 2)
    model.compile(optimizer=opt, loss='mae')
    model.fit(train_dataset, epochs=TRAINING_EPOCHS, validation_data=val_dataset,
                  verbose=1, callbacks=[callbacks, save, early_stop]) 

In [ ]:
model.load_weights(BEST_PATH)

In [ ]:
model.evaluate(val_dataset)

In [ ]:
pred_output = model.predict(val_input)

In [ ]:
pred_output = pred_output*(OUTPUT_MAXS - OUTPUT_MINS) + OUTPUT_MINS
cal_val_label = val_label*(OUTPUT_MAXS - OUTPUT_MINS) + OUTPUT_MINS

In [ ]:
fig = plt.figure(figsize=((8.5/2.54*2), (6/2.54*2)))
ax0 = plt.subplot()

ax0.spines['right'].set_visible(False)
ax0.spines['left'].set_position(('outward', 5))
ax0.spines['bottom'].set_position(('outward', 5))

ax0.plot(cal_val_label, pred_output, 'o', ms=5, mec='k', c=cmap[0])
ax0.set_ybound([0, 100])
ax0.set_xbound([0, 100])

fig.tight_layout()

In [ ]:
label_df = pd.DataFrame(cal_val_label, index=val_indices, columns=output_columns)
label_df.index = pd.DatetimeIndex(label_df.index)
pred_df = pd.DataFrame(pred_output, index=val_indices, columns=output_columns)
pred_df.index = pd.DatetimeIndex(pred_df.index)

In [ ]:
fig = plt.figure(figsize=((8.5/2.54*2), (6/2.54*2)))
ax0 = plt.subplot()

ax0.spines['right'].set_visible(False)
ax0.spines['left'].set_position(('outward', 5))
ax0.spines['bottom'].set_position(('outward', 5))

ax0.plot(pred_df.index, label_df['leaf_dw'], 'o', ms=5, mec='k', c=cmap[4])
ax0.plot(pred_df.index, pred_df['leaf_dw'], 'o', ms=5, mec='k', c=cmap[0])


fig.tight_layout()

# 예측을 해야함. 이전 값들을 받아들이는 형태로 개선해야해

In [ ]:
label_df.to_csv('./results/model_output/transformer_label.csv')
pred_df.to_csv('./results/model_output/transformer_pred.csv')